# Import any additional libraries

In [1]:
import glob
import pandas as pd
from datetime import datetime

# Extract

In [2]:
#Json extract function
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

In [3]:
columns=['Name','Market Cap (US$ Billion)']
def extract():
    extracted_data = pd.DataFrame(columns=columns)
    for jsonfile in glob.glob("*.json"):
        extracted_data = pd.concat([extracted_data, extract_from_json(jsonfile)], ignore_index=True)
    return extracted_data

In [4]:
#Load exchange_rates.csv and find the exchange rate for GBP
rates = pd.read_csv("exchange_rates.csv", index_col=0) #read csv file into dataframe
exchange_rate = rates.loc['GBP', 'Rates'] #store value of cell at row 'GBP' and column 'Rates'
exchange_rate

0.7323984208000001

# Transform

In [5]:
def transform(data, rate):
    data['Market Cap (US$ Billion)'] = data['Market Cap (US$ Billion)'].astype(float)
    #change Market Cap column from USD to GBP and round to 3 decimals
    data['Market Cap (US$ Billion)'] = round(data['Market Cap (US$ Billion)'] * rate, 3)
    #change column name to reflect change in currency
    data.rename(columns={'Market Cap (US$ Billion)':'Market Cap (GBP Billion)'}, inplace = True)
    return data

# Load

In [6]:
def load(data):
    data.to_csv('bank_market_cap_gbp.csv', index=False)

# Logging function

In [7]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ':' + message + '\n')

# Running the ETL Process


In [8]:
log("ETL Job Started")
log("Extract phase Started")
#Extract
extracted_data = extract()
extracted_data.head(5)

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


In [9]:
log("Extract phase Ended")

In [10]:
#Transform
log("Transform phase Started")
transform_data = transform(extracted_data, exchange_rate)
transform_data.head(5)

,Name,Market Cap (GBP Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


In [11]:
log("Transform phase Ended")

In [12]:
#Load
log("Load phase Started")
load(transform_data)
log("Load phase Ended")